In [73]:
import selenium
from selenium import webdriver
import time 
import re
import csv
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import math




In [91]:
driver = webdriver.Chrome(r'C:\Users\jackc\chromedriver_win32\chromedriver.exe')

## Using two random zip codes for now
zips = [20852, 10003]

## Make an empty list to store dictionaries later 
list_of_dicts = []

for element in zips:

    ## Navigate to the link based on the zipcodes provided in the list 'zips'
    link ='https://www.greatschools.org/search/search.zipcode?sort=rating&zip=' + str(element) 
    driver.get(link)

    ## Find the total number of pages per zip based on total number of schools 
    num_pages = driver.find_element_by_xpath('.//div[@class="pagination-summary"]').text
    total_schools = int(num_pages[19:21])
    num_pages = int(math.ceil(total_schools/25))

    ## For each page in each zip, loop through and pull out the relevant information
    for x in range(num_pages):
        page_num = x + 1
        if(page_num > 1):
            link = 'https://www.greatschools.org/search/search.zipcode?page=' + str(page_num) + '&sort=rating&view=list&zip='+str(element)
            driver.get(link)


        ## get a list of all of the schools on the left 
        schools = driver.find_elements_by_xpath('//li[@class=" unsaved"]')

        ## Loop through all schools 
        for school in schools: 
            name = school.find_element_by_xpath('.//a[@class="name"]').text
            address = school.find_element_by_xpath('.//div[@class="address"]').text

            ## Find level and enrollment -- they share the same tag, so separate them out
            ## Note, sometimes enrollment is not available, so check if it is and if not, make it na

            level_and_enrollment = school.find_elements_by_xpath('.//span[@class="open-sans_sb"]')
            if(len(level_and_enrollment) > 1):
                level = level_and_enrollment[0].text
                enrollment = level_and_enrollment[1].text
            else:
                level = level_and_enrollment[0].text
                enrollment = 'NA'

            rating = school.find_elements_by_xpath('.//span[@class="tipso_style"]')
            rating = rating[0].text[0]

            #If rating is unavaiable, will return 'C' -- change that to 'NA'
            if(rating == 'C'):
                rating = 'NA'

            scale = school.find_elements_by_xpath('.//div[@class="scale"]')
            scale = scale[0].text

            ## make a dictionary for each entry and add it to the list_of_dicts
            dict = {}
            dict['name'] = name
            dict['address'] = address
            dict['level'] = level
            dict['enrollment'] = enrollment
            dict['rating'] = rating
            dict['scale'] = scale
            dict['zip'] = element
            list_of_dicts.append(dict)

        # next = driver.find_elements_by_xpath('.//a[@class="anchor-button   anchor-button"]')
        # next[1].click() 

In [92]:
final_df = pd.DataFrame(list_of_dicts)